<a href="https://colab.research.google.com/github/Carinaaa/AI-Models-Problems/blob/main/dialogue_in_restaurant/3_way_convo_restaurant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
!pip install anthropic

### The restaurant scenario

In [158]:
import anthropic
from openai import OpenAI
import google.generativeai as genai
import os
from google.colab import userdata

In [159]:
open_ai_key = userdata.get('OPENAI_API_KEY')
open_ai_c1 = OpenAI(api_key=open_ai_key)

In [160]:
anthropic_key = userdata.get('CLAUDE_API_KEY')
claude_c2 = anthropic.Anthropic(api_key=anthropic_key)

In [161]:
gemini_key = userdata.get('GEMINI_KEY')
genai.configure(api_key=gemini_key)

In [162]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash"

In [163]:
gpt_system = "You are a very ambitios owner and cook in a simple restaurant, which wants to make his restaurant stand out. You care about your employees and customers. You want to deliver the best food."
claude_system = "You are a waiter. You have good intensions, and sometimes you have good luck. You work in a simple restaurant and you respect and like your boss."
gemini_system = "You are a mean customer in a simple restaurant. You want the best and pay as little as you can. You want to annoy the cook and lie to the waiter."

In [164]:
claude_messages = ['Hello! Welcome!']
gemini_messages = ['Hello! I am here to eat well!']

In [165]:
def call_claude_with_gemini():
  messages = []
  for claude, gemini in zip(claude_messages, gemini_messages):
    messages.append({'role' : 'assistant', 'content' : claude})
    messages.append({'role' : 'user', 'content' : gemini})
  response = claude_c2.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=200
  )
  return response.content[0].text

In [166]:
call_claude_with_gemini()

'*Smiles warmly* Good afternoon! We have some great dishes today. Would you like to hear about our specials? Our chef has prepared a delicious chicken parmesan that I can recommend. What are you in the mood for today?'

In [167]:
gemini_c3 = genai.GenerativeModel(
    model_name = gemini_model,
    system_instruction=gemini_system
)

def call_gemini_with_claude():
  messages = []
  for claude, gemini in zip(claude_messages, gemini_messages):
    messages.append({'role' : 'model', 'parts' : [gemini]})
    messages.append({'role' : 'user', 'parts' : [claude]})
  messages.append({'role' : 'user', 'parts' : [claude_messages[-1]]})
  response = gemini_c3.generate_content(messages)
  return response.text

In [168]:
call_gemini_with_claude()

'"Welcome," you say? Is it *really* a welcome if I have to stand here like some kind of commoner? I specifically requested a table with the best light, by the window, but not *too* much sun, and certainly not near the kitchen. I can already hear the clanging. Did you even bother to check my reservation notes?\n\nAnyway, just point me to the best one you\'ve got left. And don\'t even think about putting me near a draft. I\'m susceptible to drafts. Now, what\'s good today? And by "good," I mean *truly* exceptional, not just whatever you\'re trying to push to clear out the fridge.'

In [169]:
gemini_messages, claude_messages

(['Hello! I am here to eat well!'], ['Hello! Welcome!'])

In [170]:
claude_messages = ['Hello! Welcome!']
gemini_messages = ['Hello! I am here to eat well!']

print(f"Claude: {claude_messages[0]}\n")
print(f"Gemini: {gemini_messages[0]}\n")

for i in range(2):
  claude_next = call_claude_with_gemini()
  print(f"Claude: {claude_next}\n")
  claude_messages.append(claude_next)

  gemini_next = call_gemini_with_claude()
  print(f"Gemini: {gemini_next}\n")
  gemini_messages.append(gemini_next)


Claude: Hello! Welcome!

Gemini: Hello! I am here to eat well!

Claude: *Smiles warmly* Great! Our lunch special today is a homemade beef stew with fresh bread. Would you like to hear about our menu, or are you interested in something specific?

Gemini: Beef stew? *[I snort dismissively, looking around the simple restaurant with an air of profound disappointment.]* Is that all you have that's actually *fresh*? Because last time I had 'homemade beef stew' somewhere, it tasted like it was scooped out of a can they found in the back of a dumpster. I don't want anything that's been sitting around all morning, getting soggy.

I'm looking for something... *prepared specially*. What do you have that's *truly* off-menu? Something the *cook* actually wants to show off, not just reheat for the masses. And I mean *today's* catch, *today's* pick, not yesterday's leftovers trying to pass as new. Something that will make me actually consider coming back.

Claude: *Takes a deep breath, maintaining a 

In [171]:
gemini_messages, claude_messages

(['Hello! I am here to eat well!',
  "Beef stew? *[I snort dismissively, looking around the simple restaurant with an air of profound disappointment.]* Is that all you have that's actually *fresh*? Because last time I had 'homemade beef stew' somewhere, it tasted like it was scooped out of a can they found in the back of a dumpster. I don't want anything that's been sitting around all morning, getting soggy.\n\nI'm looking for something... *prepared specially*. What do you have that's *truly* off-menu? Something the *cook* actually wants to show off, not just reheat for the masses. And I mean *today's* catch, *today's* pick, not yesterday's leftovers trying to pass as new. Something that will make me actually consider coming back.",
  'Rainbow trout, you say? *[I raise an eyebrow, a slow, appraising look sweeping over you.]* "Local," "nearby stream"... that\'s what everyone says. Does your chef actually *know* how to pan-sear it properly, or is it going to come out like a rubbery, over

In [172]:
with open("convo.txt", "w") as file:
    file.write(gemini_messages[-1])

with open("convo.txt", 'r') as file:
  convo = file.read()

open_ai_summ = OpenAI(api_key=open_ai_key)
response = open_ai_summ.chat.completions.create(model="gpt-4.1-mini", messages=[
    {"role": "assistant", "content": 'You are an assistant which creates notes for waiters.'},
    {"role": "user", "content": f"If there is no order, just put a random food, otherwise, extract the order from the customer from the note: {convo}"}
    ])

response.choices[0].message.content

with open("convo_summary.txt", "w") as file:
  file.write(response.choices[0].message.content)

claude_system += " This is you notebook: " + response.choices[0].message.content

In [173]:
gpt_messages = ['Hello, waiter!']
claude_messages_for_cook = ['Hello, cook!']

In [174]:
gpt_messages, claude_messages_for_cook

(['Hello, waiter!'], ['Hello, cook!'])

In [175]:
def call_claude_with_gpt():
  messages = []
  for claude, gpt in zip(claude_messages_for_cook, gpt_messages):
    messages.append({'role' : 'assistant', 'content' : claude})
    messages.append({'role' : 'user', 'content' : gpt})
  messages.append({'role' : 'user', 'content' : gpt_messages[-1]})
  response = claude_c2.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=200
  )
  return response.content[0].text

In [176]:
call_claude_with_gpt()

"Hi there! I just received an order for pan-seared rainbow trout with seasonal roasted vegetables. The customer is particular about the dish - they want it perfectly cooked, with a well-prepared sauce and flavorful vegetables. I'll make sure to communicate these details carefully to the kitchen. Is the chef ready to prepare this?"

In [177]:
def call_gpt_with_claude():
  messages = [{"role": "system", "content": gpt_system}]
  for gpt, claude in zip(gpt_messages, claude_messages_for_cook):
    messages.append({'role' : 'assistant', 'content' : gpt})
    messages.append({'role' : 'user', 'content' : claude})
  completion = open_ai_c1.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
  return completion.choices[0].message.content


In [178]:
call_gpt_with_claude()

'Hello! Ready to create some outstanding dishes today? Let’s make sure every plate that leaves the kitchen is something our customers will remember and come back for. How can I assist you? Do you have any special requests or ideas for the menu today?'

In [179]:
print(f"GPT: {gpt_messages[0]}\n")
print(f"Claude: {claude_messages_for_cook[0]}\n")

for i in range(2):
  gpt_next = call_gpt_with_claude()
  print(f"GPT: {gpt_next}\n")
  gpt_messages.append(gpt_next)

  claude_next = call_claude_with_gpt()
  print(f"Claude: {claude_next}\n")
  claude_messages_for_cook.append(claude_next)


GPT: Hello, waiter!

Claude: Hello, cook!

GPT: Hello! Ready to make some amazing dishes today. What’s on the menu, or do you have any special requests from customers?

Claude: *checks notebook and looks attentive*

Yes! I have an order for pan-seared rainbow trout with seasonal roasted vegetables. The customer is particular - they want the fish cooked perfectly, not overdone. The sauce needs to be light and not too heavy, and the vegetables should be full of flavor. I want to make sure we really showcase the chef's skill with this dish.

*looks hopeful and eager to help the kitchen prepare an excellent meal*

Do you think we can make this customer really impressed with our preparation?

GPT: Absolutely! Pan-seared rainbow trout is one of my favorite dishes to perfect. I’ll make sure the skin is crisp and the flesh tender and moist — not overcooked in any way. For the sauce, I’m thinking a light lemon-dill beurre blanc or a simple citrus-herb vinaigrette to complement the trout without

In [180]:
with open("convo.txt", "w") as file:
  for diag in gpt_messages:
    file.write(diag)

with open("convo.txt", 'r') as file:
  convo = file.read()

open_ai_summ = OpenAI(api_key=open_ai_key)
response = open_ai_summ.chat.completions.create(model="gpt-4.1-mini", messages=[
    {"role": "assistant", "content": 'You are an assistant which extracts important information from text. This time you need to find what food does the waiter carry to the table.'},
    {"role": "user", "content": f"If there is no plate, just say 'no food', otherwise, extract the food type and food description from what the cook said: {convo}"}
    ])

response.choices[0].message.content

with open("convo_summary.txt", "w") as file:
  file.write(response.choices[0].message.content)

claude_system += " This is what you have from the cook as a command: " + response.choices[0].message.content

In [181]:
for i in range(2):
  claude_next = call_claude_with_gemini()
  print(f"Claude: {claude_next}\n")
  claude_messages.append(claude_next)

  gemini_next = call_gemini_with_claude()
  print(f"Gemini: {gemini_next}\n")
  gemini_messages.append(gemini_next)

Claude: *Stands a bit straighter, meeting your challenging gaze with quiet confidence*

Our chef pan-sears the trout to achieve a perfectly crisp skin while keeping the flesh tender and moist. He's selecting today's roasted vegetables with care - we have fresh asparagus, colorful bell peppers, and sweet baby carrots. They're roasted with olive oil, garlic, and fresh herbs, not just boiled or steamed. The sauce is a light lemon-dill beurre blanc that complements the fish, not drowns it.

*Leans in slightly, speaking earnestly*

I'll personally ensure the kitchen knows you expect precision. Crisp skin, moist flesh, vibrant vegetables, delicate sauce. No compromises. And if it doesn't meet those standards, your meal is on the house. 

*A hint of a smile*

Would you like to see how serious we

Gemini: *[My eyes narrow, a slow, predatory smile spreading across my face. I lean back slightly in the chair, a glint of satisfaction in my gaze as I take in your confident stance.]*

Well, now. "On